## Bayesian Model Setup


In [20]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

file_path = '/content/drive/My Drive/marketing_data.csv'
df = pd.read_csv(file_path)
print(df.columns)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Index(['start_of_week', 'revenue', 'spend_channel_1', 'spend_channel_2',
       'spend_channel_3', 'spend_channel_4', 'spend_channel_5',
       'spend_channel_6', 'spend_channel_7', 'spend_channel_1_adstocked',
       'spend_channel_2_adstocked', 'spend_channel_3_adstocked',
       'spend_channel_4_adstocked', 'spend_channel_5_adstocked',
       'spend_channel_6_adstocked', 'spend_channel_7_adstocked'],
      dtype='object')


In [21]:
df.head()

,start_of_week,revenue,spend_channel_1,spend_channel_2,spend_channel_3,spend_channel_4,spend_channel_5,spend_channel_6,spend_channel_7,spend_channel_1_adstocked,spend_channel_2_adstocked,spend_channel_3_adstocked,spend_channel_4_adstocked,spend_channel_5_adstocked,spend_channel_6_adstocked,spend_channel_7_adstocked
0,2020-08-30,157906.75,2625.48,262.71,12954.12,3609.63,12955.29,12659.12,19379.79,2625.480000,262.710000,12954.120000,3609.630000,12955.290000,12659.120000,19379.790000
1,2020-09-06,186425.68,2634.01,108.66,8760.28,4560.60,12747.70,12338.18,22473.45,2896.558000,345.099000,10055.692000,7809.267000,14043.229000,14870.004000,24411.429000
2,2020-09-13,161607.39,2087.08,110.32,7155.42,4362.96,15015.41,10811.15,22596.05,2376.735800,420.909100,8160.989200,11391.300300,16419.732900,13785.150800,25037.192900
3,2020-09-20,180089.13,1690.70,52.79,15185.22,3883.41,15521.41,12890.22,24728.73,1928.373580,431.608190,16001.318920,14135.580270,17163.383290,15647.250160,27232.449290
4,2020-09-27,217793.98,1547.30,80.56,18524.05,4043.09,15793.74,12642.55,26515.48,1740.137358,469.007371,20124.181892,16765.112243,17510.078329,15772.000032,29238.724929


### Define the Model


In [103]:
import pymc as pm
import numpy as np
import pytensor.tensor as pt
import pandas as pd

# Number of marketing channels
num_channels = 7

# number of data points
n_weeks = df.shape[0]

adstocked_spends = df[['spend_channel_1_adstocked', 'spend_channel_2_adstocked',
                       'spend_channel_3_adstocked', 'spend_channel_4_adstocked',
                       'spend_channel_5_adstocked', 'spend_channel_6_adstocked',
                       'spend_channel_7_adstocked']].values

revenue = df['revenue'].values

# time indices for the trend component
time_index = np.arange(n_weeks)

# Create seasonal components (assuming weekly data with yearly seasonality)
# We'll use Fourier terms to capture seasonality - here with 52 weeks per year
n_harmonics = 5  # increased from 4 to 5 for better seasonality modeling
fourier_features = np.zeros((n_weeks, 2 * n_harmonics))
for i in range(n_harmonics):
    fourier_features[:, 2*i] = np.sin(2 * np.pi * (i+1) * time_index / 52)
    fourier_features[:, 2*i+1] = np.cos(2 * np.pi * (i+1) * time_index / 52)

# Month indicator variables
january_indicator = (pd.to_datetime(df['start_of_week']).dt.month == 1).astype(int)
november_indicator = (pd.to_datetime(df['start_of_week']).dt.month == 11).astype(int)
december_indicator = (pd.to_datetime(df['start_of_week']).dt.month == 12).astype(int)

# PyMC model
with pm.Model() as model:

    baseline = pm.Normal('baseline', mu=np.mean(revenue), sigma=np.std(revenue))

    # Prior for the trend component
    trend = pm.Normal('trend', mu=0, sigma=1)

    # Priors for seasonal components
    seasonal_coef = pm.Normal('seasonal_coef', mu=0, sigma=5, shape=2*n_harmonics)

    # Priors for the channel coefficients (betas)
    betas = pm.Normal('betas', mu=0, sigma=10, shape=num_channels)

    # Priors for holiday effects
    january_effect = pm.Normal('january_effect', mu=0, sigma=50000)
    november_effect = pm.Normal('november_effect', mu=0, sigma=40000)  # Added November effect
    december_effect = pm.Normal('december_effect', mu=0, sigma=30000)

    # Prior for the error term (sigma), assuming positive uncertainty
    sigma = pm.HalfNormal('sigma', sigma=10)

    # Combine components to get expected revenue
    trend_component = trend * time_index
    seasonal_component = pm.math.dot(fourier_features, seasonal_coef)
    marketing_component = pm.math.dot(adstocked_spends, betas)

    # Updated holiday component with November
    holiday_component = (january_effect * january_indicator +
                        november_effect * november_indicator +
                        december_effect * december_indicator)

    mu = baseline + trend_component + seasonal_component + marketing_component + holiday_component

    # Likelihood of the observed data (revenue), assuming normally distributed errors
    likelihood = pm.Normal('revenue', mu=mu, sigma=sigma, observed=revenue)

    # Sampling from the posterior distribution
    trace = pm.sample(1000, tune=1000, return_inferencedata=True)

Output()

## Model Parameter Estimates

In [104]:
# arviz for analyzing the results
import arviz as az

# summary of the posterior distributions
summary = az.summary(trace, round_to=3)
print(summary)

# marketing channel coefficients
channel_summary = summary.filter(like='betas', axis=0)
print("\nMarketing Channel Coefficients:")
print(channel_summary)

                       mean        sd     hdi_3%    hdi_97%  mcse_mean  \
baseline          74705.131  1092.607  72541.049  76677.054     24.920   
betas[0]             -4.977     0.181     -5.329     -4.656      0.003   
betas[1]             -1.832     0.123     -2.050     -1.581      0.002   
betas[2]              0.865     0.026      0.815      0.912      0.001   
betas[3]             -0.274     0.015     -0.300     -0.246      0.000   
betas[4]              1.541     0.044      1.452      1.618      0.001   
betas[5]              2.050     0.036      1.980      2.114      0.001   
betas[6]              1.244     0.019      1.207      1.279      0.000   
december_effect  -18403.309   799.846 -19829.335 -16871.674     15.898   
january_effect    31347.015   656.373  30127.884  32572.840     12.298   
november_effect   60997.314   694.349  59719.752  62261.371     11.932   
seasonal_coef[0]      0.059     4.752     -8.450      9.067      0.081   
seasonal_coef[1]     -1.977     5.153 

### Creating a more readable table for the channel coefficients

In [105]:
import pandas as pd

channel_names = ['spend_channel_1', 'spend_channel_2', 'spend_channel_3',
                'spend_channel_4', 'spend_channel_5', 'spend_channel_6',
                'spend_channel_7']

channel_df = pd.DataFrame({
    'Channel': channel_names,
    'Coefficient': summary.filter(like='betas', axis=0)['mean'].values,
    'Lower Bound (3%)': summary.filter(like='betas', axis=0)['hdi_3%'].values,
    'Upper Bound (97%)': summary.filter(like='betas', axis=0)['hdi_97%'].values
})

# rating
def get_effectiveness(coef):
    if coef > 1:
        return "Very Positive"
    elif coef > 0.5:
        return "Positive"
    elif coef > 0:
        return "Slightly Positive"
    elif coef > -0.5:
        return "Slightly Negative"
    elif coef > -1:
        return "Negative"
    else:
        return "Very Negative"

channel_df['Impact'] = channel_df['Coefficient'].apply(get_effectiveness)

# Sorting by coefficient value
sorted_channels = channel_df.sort_values('Coefficient', ascending=False)
print(sorted_channels)

           Channel  Coefficient  Lower Bound (3%)  Upper Bound (97%)  \
5  spend_channel_6        2.050             1.980              2.114   
4  spend_channel_5        1.541             1.452              1.618   
6  spend_channel_7        1.244             1.207              1.279   
2  spend_channel_3        0.865             0.815              0.912   
3  spend_channel_4       -0.274            -0.300             -0.246   
1  spend_channel_2       -1.832            -2.050             -1.581   
0  spend_channel_1       -4.977            -5.329             -4.656   

              Impact  
5      Very Positive  
4      Very Positive  
6      Very Positive  
2           Positive  
3  Slightly Negative  
1      Very Negative  
0      Very Negative  


In [106]:
import pandas as pd

channel_names = ['Channel 1', 'Channel 2', 'Channel 3', 'Channel 4',
                'Channel 5', 'Channel 6', 'Channel 7']

channel_df = pd.DataFrame({
    'Channel': channel_names,
    'Coefficient': summary.filter(like='betas', axis=0)['mean'].values,
    'Lower Bound (3%)': summary.filter(like='betas', axis=0)['hdi_3%'].values,
    'Upper Bound (97%)': summary.filter(like='betas', axis=0)['hdi_97%'].values
})


def get_effectiveness(coef):
    if coef > 1:
        return "Very Positive"
    elif coef > 0.5:
        return "Positive"
    elif coef > 0:
        return "Slightly Positive"
    elif coef > -0.5:
        return "Slightly Negative"
    elif coef > -1:
        return "Negative"
    else:
        return "Very Negative"

# effectiveness function to 'Coefficient'
channel_df['Impact'] = channel_df['Coefficient'].apply(get_effectiveness)

sorted_channels = channel_df.sort_values('Coefficient', ascending=False)

styled_table = sorted_channels.style.format({
    'Coefficient': '{:.3f}',
    'Lower Bound (3%)': '{:.3f}',
    'Upper Bound (97%)': '{:.3f}'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#4CAF50'),
                                ('color', 'white'),
                                ('font-weight', 'bold'),
                                ('font-size', '14px')]},
    {'selector': 'td', 'props': [('border', '1px solid black'),
                                 ('padding', '8px'),
                                 ('font-size', '12px'),
                                 ('color', 'black')]},
    {'selector': 'tr:nth-child(odd)', 'props': [('background-color', '#f2f2f2')]},
    {'selector': 'tr:nth-child(even)', 'props': [('background-color', 'white')]},
    {'selector': 'td:hover', 'props': [('background-color', '#e2e2e2')]},
])

styled_table.hide(axis="index")
styled_table


Channel,Coefficient,Lower Bound (3%),Upper Bound (97%),Impact
Channel 6,2.050,1.980,2.114,Very Positive
Channel 5,1.541,1.452,1.618,Very Positive
Channel 7,1.244,1.207,1.279,Very Positive
Channel 3,0.865,0.815,0.912,Positive
Channel 4,-0.274,-0.300,-0.246,Slightly Negative
Channel 2,-1.832,-2.050,-1.581,Very Negative
Channel 1,-4.977,-5.329,-4.656,Very Negative


### Checking model predictions against actual data

In [107]:
with model:
    posterior_pred = pm.sample_posterior_predictive(trace)

# Get predicted values
pred_mean = posterior_pred.posterior_predictive['revenue'].mean(dim=["chain", "draw"]).values

print(pred_mean[:10])  # first 10 values


Output()

[141321.36758471 148646.11527389 150830.24374288 166737.68233409
 173692.09193037 199279.91050556 238391.62188311 203593.68023045
 157515.46363978 207410.33812473]


In [108]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['start_of_week'],
    y=df['revenue'],
    mode='markers',
    name='Actual Revenue',
    marker=dict(size=8, opacity=0.6, color='blue')
))

fig.update_layout(
    title='Actual Revenue Over Time',
    xaxis_title='Date',
    yaxis_title='Revenue',
    template='plotly_white',
    width=900,
    height=500
)

# Show the figure
fig.show()


### Predicted revenue to the existing plot

In [109]:
fig.add_trace(go.Scatter(
    x=df['start_of_week'],
    y=pred_mean,
    mode='lines',
    name='Predicted Revenue',
    line=dict(width=2, color='red')
))

fig.update_layout(
    title='Model Fit: Actual vs Predicted Revenue',
    xaxis_title='Date',
    yaxis_title='Revenue',
    hovermode='x unified',
    template='plotly_white',
    width=900,
    height=500
)

fig.show()


### Calculating R-squared to Evaluate Model Fit

In [110]:
# Calculate R-squared
ss_total = np.sum((df['revenue'] - np.mean(df['revenue']))**2)
ss_residual = np.sum((df['revenue'] - pred_mean)**2)
r_squared = 1 - (ss_residual / ss_total)

# Update layout and add R-squared annotation
fig.update_layout(
    title=f'Model Fit: Actual vs Predicted Revenue (R² = {r_squared:.3f})',
    xaxis_title='Date',
    yaxis_title='Revenue',
    hovermode='x unified',
    template='plotly_white',
    width=900,
    height=500
)

# Show the final figure
fig.show()


### Calculating ROI for Each Channel

In [111]:
# Calculate ROI for each channel
channel_names = ['Channel 1', 'Channel 2', 'Channel 3',
                'Channel 4', 'Channel 5', 'Channel 6', 'Channel 7']
original_channels = ['spend_channel_1', 'spend_channel_2', 'spend_channel_3',
                    'spend_channel_4', 'spend_channel_5', 'spend_channel_6', 'spend_channel_7']
adstocked_channels = [ch + '_adstocked' for ch in original_channels]

# Get coefficients
coefficients = trace.posterior['betas'].mean(dim=('chain', 'draw')).values

# Calculate total spend and contribution for each channel
total_spend = []
total_contribution = []

for i, channel in enumerate(original_channels):
    # Total spend
    spend = df[channel].sum()
    total_spend.append(spend)

    # Total contribution to revenue (coefficient * adstocked_spend)
    contribution = coefficients[i] * df[adstocked_channels[i]].sum()
    total_contribution.append(contribution)

# Calculate ROI (Return on Investment)
roi = [(contribution / spend) for contribution, spend in zip(total_contribution, total_spend)]

# Create DataFrame
roi_df = pd.DataFrame({
    'Channel': channel_names,
    'Coefficient': coefficients,
    'Total Spend': total_spend,
    'Total Contribution': total_contribution,
    'ROI': roi
})

# Sort by ROI
roi_df = roi_df.sort_values('ROI', ascending=False)

print(roi_df)

     Channel  Coefficient  Total Spend  Total Contribution        ROI
5  Channel 6     2.049660    526624.70        1.349252e+06   2.562076
4  Channel 5     1.541210    891863.59        1.526708e+06   1.711817
6  Channel 7     1.243523   2880942.21        3.975915e+06   1.380074
2  Channel 3     0.865302   2028746.51        1.949380e+06   0.960879
3  Channel 4    -0.273732    719174.22       -1.816454e+06  -2.525750
0  Channel 1    -4.977340    129542.90       -7.164212e+05  -5.530378
1  Channel 2    -1.832137     35738.66       -5.505618e+05 -15.405217


In [112]:
import plotly.express as px
import pandas as pd

# Plot ROI by Channel
fig = px.bar(
    roi_df,
    x='Channel',
    y='ROI',
    text='ROI',
    color='ROI',
    color_continuous_scale=['grey', 'lightgreen', 'darkgreen'],
    title="ROI by Channel"
)

# Adjust text and layout for better visibility
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(
    yaxis_title="ROI",
    xaxis_title="Channel",
    coloraxis_showscale=False,  # Hide color scale
    template="plotly_white",
    height=600,
    width=900
)

fig.show()


#### Channel 6's spend is relatively low, but the revenue generated from that spend is high — leading to a high ROI. each euro spent is associated with a €2.56 profit

#### Channel 2 has the lowest ROI (-15.41) - each euro spent is associated with a €-15.41 loss

In [113]:
# Plot Total Contribution by Channel
fig = px.bar(
    roi_df,
    x='Channel',
    y='Total Contribution',
    text='Total Contribution',
    color='Total Contribution',
    color_continuous_scale=['grey', 'lightgreen', 'darkgreen'],
    title="Total Contribution by Channel"
)

# Adjust text and layout for better visibility
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(
    yaxis_title="Contribution (€)",
    xaxis_title="Channel",
    coloraxis_showscale=False,  # Hide color scale
    template="plotly_white",
    height=600,
    width=900
)

fig.show()


#### Channel 7 has the highest total revenue contribution €3.98M, indicating that it’s driving significant absolute revenue.

However, its ROI is 1.38, which suggests that while it's effective in generating revenue, the efficiency (revenue per € spent) is moderate compared to Channel 6

In [114]:
def calculate_mape(actual, predicted):
    mape = (abs(actual - predicted) / (actual + 1e-10)).mean() * 100
    return mape


In [115]:
actual = df['revenue'].values
print(f"Actual values: {actual[:5]}")  # Show the first 5 actual values


Actual values: [157906.75 186425.68 161607.39 180089.13 217793.98]


In [116]:
with model:
    posterior_pred = pm.sample_posterior_predictive(trace, var_names=['revenue'], random_seed=42)

print(posterior_pred.posterior_predictive['revenue'].shape)


Output()

(2, 1000, 104)


In [117]:
predicted = posterior_pred.posterior_predictive['revenue'].mean(dim=["chain", "draw"]).values
print(f"Predicted values: {predicted[:5]}")

Predicted values: [141398.29391589 148700.70156074 150714.45938788 166721.3319167
 173725.43853753]


In [118]:
mape = calculate_mape(actual, predicted)
print(f"MAPE: {mape:.2f}%")


MAPE: 17.34%


In [119]:
# Calculate Mean Absolute Error (MAE)
def calculate_mae(actual, predicted):
    mae = abs(actual - predicted).mean()
    return mae

# Using your actual and predicted values
actual = df['revenue'].values
predicted = posterior_pred.posterior_predictive['revenue'].mean(dim=["chain", "draw"]).values

# Compute MAE
mae = calculate_mae(actual, predicted)
print(f"MAE: €{mae:.2f}")

MAE: €23079.03


In [120]:
from plotly.subplots import make_subplots
# Extract components from the model
baseline_mean = trace.posterior['baseline'].mean().values
trend_mean = trace.posterior['trend'].mean().values
seasonal_coef_mean = trace.posterior['seasonal_coef'].mean(dim=["chain", "draw"]).values

# Calculate components
trend_component = trend_mean * time_index
seasonal_component = np.dot(fourier_features, seasonal_coef_mean)
january_effect = trace.posterior['january_effect'].mean().values * january_indicator
december_effect = trace.posterior['december_effect'].mean().values * december_indicator
holiday_component = january_effect + december_effect

# Create subplots
fig = make_subplots(rows=4, cols=1,
                    subplot_titles=("Total Revenue", "Trend Component",
                                   "Seasonal Component", "Holiday Effect Component"))

# Add total revenue
fig.add_trace(
    go.Scatter(x=df['start_of_week'], y=revenue, mode='lines+markers',
              name='Revenue', line=dict(color='blue')),
    row=1, col=1
)

# Add trend component
fig.add_trace(
    go.Scatter(x=df['start_of_week'], y=trend_component, mode='lines',
              name='Trend', line=dict(color='red')),
    row=2, col=1
)

# Add seasonal component
fig.add_trace(
    go.Scatter(x=df['start_of_week'], y=seasonal_component, mode='lines',
              name='Seasonality', line=dict(color='green')),
    row=3, col=1
)

# Add holiday component
fig.add_trace(
    go.Scatter(x=df['start_of_week'], y=holiday_component, mode='lines',
              name='Holiday Effects', line=dict(color='purple')),
    row=4, col=1
)

# Update layout
fig.update_layout(height=800, width=1000,
                 title_text="Revenue Decomposition: Trend, Seasonality, and Holiday Effects",
                 showlegend=False)

fig.update_yaxes(title_text="Revenue (€)", row=1, col=1)
fig.update_yaxes(title_text="Trend (€)", row=2, col=1)
fig.update_yaxes(title_text="Seasonal Effect (€)", row=3, col=1)
fig.update_yaxes(title_text="Holiday Effect (€)", row=4, col=1)

fig.show()